Other resources:
- https://itnext.io/prometheus-building-a-custom-prometheus-exporter-in-python-988908327600
- https://matthewzhaocc.com/building-a-custom-prometheus-exporter-in-python-6491d4cdcef3

## Always-up Metrics Endpoint:

This code hosts Prometheus-compatible dummy metrics on the localhost. These can then be queried by Prometheus (if Prometheus can access the localhost).
The purpose of this is to test that Prometheus is able to reach these metrics and parse / analyse them correctly.

In [1]:
import time
import random
from os import path
import yaml
from prometheus_client.core import GaugeMetricFamily, REGISTRY, CounterMetricFamily
from prometheus_client import start_http_server
totalRandomNumber = 0
class RandomNumberCollector(object):
    def __init__(self):
        pass
    def collect(self):
        gauge = GaugeMetricFamily("random_number", "A random number generator, I have no better idea", labels=["randomNum"])
        gauge.add_metric(['random_num'], random.randint(1, 20))
        yield gauge
        count = CounterMetricFamily("random_number_2", "A random number 2.0", labels=['randomNum'])
        global totalRandomNumber
        totalRandomNumber += random.randint(1,30)
        count.add_metric(['random_num'], totalRandomNumber)
        yield count

#======================================
if __name__ == "__main__":
    port = 9000
    start_http_server(port)
    REGISTRY.register(RandomNumberCollector())
    while True: 
        # period between collection
        time.sleep(1)


## Python Pushgateway:

This code is a Python-based exporter of Prometheus-compatible metrics designed to work with the Prometheus "Push Gateway".
The Prometheus Push Gateway is a separate service that integrates with Prometheus to allow metrics to be *pushed* to Prometheus rather than queried by Prometeus.

https://prometheus.github.io/client_python/exporting/pushgateway/

In [12]:
from prometheus_client import CollectorRegistry, Gauge, push_to_gateway
from prometheus_client.exposition import basic_auth_handler

url = "https://bi-monitoring.mydomain.com/pushgateway"

def my_auth_handler(url, method, timeout, headers, data):
    username = "admin"
    password = "<< PROMETHEUS PUSHGATEWAY BASIC AUTH PASSWORD HERE >>"
    return basic_auth_handler(url, method, timeout, headers, data, username, password)

registry = CollectorRegistry()
metrics = [
    {
        "name":"errors_jira_integration_script",
        "documentation":"The number of errors recorded in the Jira Integration script",
        "observation":0
    },
    {
        "name":"warnings_jira_integration_script",
        "documentation":"The number of warnings recorded in the Jira Integration script",
        "observation":0
    }
]

for met in metrics:
    g = Gauge(name=met['name'],documentation=met['documentation'],registry=registry)
    g.set(met['observation'])
    push_to_gateway(url, job='jira-integration', registry=registry, handler=my_auth_handler)

In [1]:
from prometheus_client import CollectorRegistry, Gauge, push_to_gateway
from prometheus_client.exposition import basic_auth_handler

url = "https://bi-monitoring.mydomain.com/pushgateway"

def my_auth_handler(url, method, timeout, headers, data):
    username = "admin"
    password = "<< PROMETHEUS PUSHGATEWAY BASIC AUTH PASSWORD HERE >>"
    return basic_auth_handler(url, method, timeout, headers, data, username, password)

registry = CollectorRegistry()
metrics = [
    {
        "name":"errors_git_repo_migrator",
        "documentation":"Last reported number of errors from the Git Repo Migrator batch script",
        "observation":0
    }
]

for met in metrics:
    g = Gauge(name=met['name'],documentation=met['documentation'],registry=registry)
    g.set(met['observation'])
    push_to_gateway(url, job='git-repo-migrator', registry=registry, handler=my_auth_handler)